# Imports and clone 'MRI' generation repository from GitHub

11.50 started

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')
!pip install keras_tuner 
import keras_tuner as kt
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras import Model
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt

!git clone https://github.com/JamesCallanan/Grad-CAM.git

%cd /content/Grad-CAM

from MRIDetails import EXPERIMENT_MODE, disease_details, SEG_MASK_KEYS, get_segment_seg_maps
from db_funcs import Experiment_Number, insert_search, insert_trial

import subprocess

def get_git_revision_short_hash() -> str:
    return subprocess.check_output(['git', 'rev-parse', '--short', 'HEAD']).decode('ascii').strip()

def get_git_branch() -> str:
    return subprocess.check_output(['git','rev-parse','--abbrev-ref','HEAD']).decode('ascii').strip().split('* ')[-1]

current_commit_id = get_git_revision_short_hash()

%cd /content/


Mounted at /content/gdrive/
     |████████████████████████████████| 133 kB 5.3 MB/s 
Cloning into 'Grad-CAM'...
remote: Enumerating objects: 126, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 126 (delta 68), reused 80 (delta 32), pack-reused 0
Receiving objects: 100% (126/126), 25.53 KiB | 1.42 MiB/s, done.
Resolving deltas: 100% (68/68), done.
/content/Grad-CAM
/content


/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


# Edit these!

In [ ]:
import os
import uuid
experiment_number = Experiment_Number.EXP_1_NORMAL_LOSS.value
json_file = "/content/gdrive/MyDrive/ME Project/New idea/Results/Experiment 1/Normal loss/searches.json"

search_id = str(uuid.uuid4())
search_dir = f'/content/gdrive/MyDrive/ME Project/New idea/Results/Experiment 1/Normal loss/{search_id}/'

num_models = 1
num_epochs = 6

#started 18:34
database_connection_details = {
    'ngrok_host' : '8.tcp.ngrok.io',
    'ngrok_port' : '14545',
    'user' : 'james'
}

data_dir = '/content/gdrive/MyDrive/ME Project/Data/Exp1 - 95a8c3d'

exp1_train_mris_unprocessed   = np.load(os.path.join(data_dir,'exp1_train_mris_unprocessed.npy'))
exp1_train_mris_preprocessed  = np.load(os.path.join(data_dir,'exp1_train_mris_preprocessed.npy'))
exp1_train_seg_masks          = np.load(os.path.join(data_dir,'exp1_train_seg_masks.npy'))
exp1_train_label              = np.load(os.path.join(data_dir,'exp1_train_labels.npy'))

exp1_val_mris_unprocessed     = np.load(os.path.join(data_dir,'exp1_val_mris_unprocessed.npy'))
exp1_val_mris_preprocessed    = np.load(os.path.join(data_dir,'exp1_val_mris_preprocessed.npy'))
exp1_val_seg_masks            = np.load(os.path.join(data_dir,'exp1_val_seg_masks.npy'))
exp1_val_label                = np.load(os.path.join(data_dir,'exp1_val_labels.npy')) 


# Dummy data 
# data_dir = '/content/gdrive/MyDrive/ME Project/Data/Dummy data'

# exp1_train_mris_unprocessed   = np.load(os.path.join(data_dir,'dummy_train_mris_unprocessed.npy'))
# exp1_train_mris_preprocessed  = np.load(os.path.join(data_dir,'dummy_train_mris_preprocessed.npy'))
# exp1_train_seg_masks          = np.load(os.path.join(data_dir,'dummy_train_seg_masks.npy'))
# exp1_train_label              = np.load(os.path.join(data_dir,'dummy_train_labels.npy'))

# exp1_val_mris_unprocessed     = np.load(os.path.join(data_dir,'dummy_val_mris_unprocessed.npy'))
# exp1_val_mris_preprocessed    = np.load(os.path.join(data_dir,'dummy_val_mris_preprocessed.npy'))
# exp1_val_seg_masks            = np.load(os.path.join(data_dir,'dummy_val_seg_masks.npy'))
# exp1_val_label                = np.load(os.path.join(data_dir,'dummy_val_labels.npy')) 

hyperparam_ranges = {
    'lr': {
            'min_val': 0.00005,
            'max_val': 0.00005 
          }
}

disease_classes = ['NOR', 'RV', 'HCM', 'DCM']
num_output_neurons = len(disease_classes)

# Load data and perform smell test on it

In [ ]:
def print_summary(ar):
  print('Max ',np.amax(ar))
  print('Min ',np.amin(ar))
  print('Mean ',np.mean(ar))
  
print('exp1_train_mris_preprocessed')
print_summary(exp1_train_mris_preprocessed)
print('exp1_val_mris_preprocessed')
print_summary(exp1_val_mris_preprocessed)

exp1_train_mris_preprocessed
Max  0.8745098
Min  -1.0
Mean  -0.8466242
exp1_val_mris_preprocessed
Max  0.8666667
Min  -1.0
Mean  -0.8462067


In [ ]:
print(search_dir)

/content/gdrive/MyDrive/ME Project/New idea/Results/Experiment 1/Normal loss/0c1420a1-af54-4101-9182-e741fe936b72/


In [ ]:
tuner_log_filepath = os.path.join(search_dir, 'tuner_search_logs')
os.makedirs(search_dir)
model_folder_path = os.path.join(search_dir, 'Models/')
os.makedirs(model_folder_path)
os.path.join(search_dir, 'tb/')
tensorboard_folder_path = os.path.join(search_dir, 'tb/')
keras_tuner_path = os.path.join(search_dir, 'kt/')

def get_model(lr):
  model = VGG16(include_top=False, input_shape=(105, 135,3))
  for layer in model.layers:
    layer.trainable = False
  x = Flatten()(model.layers[-1].output)
  x = Dense(100, activation='relu', kernel_initializer='he_uniform')(x)  
  logits = Dense(num_output_neurons, activation=None)(x)
  output = tf.keras.activations.sigmoid(logits)
  model = Model(inputs=model.inputs, outputs=output)
  opt = Adam(learning_rate=lr)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy', 'categorical_crossentropy'])
  return model
  
def build_model(hp):
    lr = hp.Float("lr", min_value=hyperparam_ranges['lr']['min_val'], max_value=hyperparam_ranges['lr']['max_val'], sampling="log")
    model1 = get_model(lr=lr )
    return model1

tuner2 = kt.BayesianOptimization(
    hypermodel=build_model,
    objective= 'val_loss',
    max_trials=num_models,
    executions_per_trial=1,
    overwrite=True,
    directory= keras_tuner_path,
    project_name="keras_tuner",
)
tuner2.search(x = exp1_train_mris_preprocessed, y=exp1_train_label , validation_data=(exp1_val_mris_preprocessed, exp1_val_label), batch_size=1, epochs=num_epochs, callbacks=[tf.keras.callbacks.TensorBoard(tensorboard_folder_path)])


Trial 1 Complete [00h 24m 25s]
val_loss: 0.00031379490974359214

Best val_loss So Far: 0.00031379490974359214
Total elapsed time: 00h 24m 25s
INFO:tensorflow:Oracle triggered exit


In [ ]:
import json
import os

if os.path.isfile(json_file):
  a_file = open(json_file, "r")
  json_object = json.load(a_file)
  a_file.close()

  list_of_string_search_numbers = list(json_object.keys())
  list_of_integer_search_numbers = list(map(int, list_of_string_search_numbers))

  max_search_number = max(list_of_integer_search_numbers)
  current_search_number = max_search_number + 1

else:
  current_search_number = 0
  json_object = {}

json_object[current_search_number] = search_id

a_file = open(json_file, "w")
json.dump(json_object, a_file)
a_file.close()


# The following would be imported if we had the correct git commit

In [ ]:
tuner_search = {
                  'search_id' :  search_id, 
                  'experiment_number' : experiment_number,
                  'num_models' :  num_models,
                  'num_epochs' :  num_epochs,
                  'hyperparam_ranges' :  str(hyperparam_ranges),
                  'git_commit_id' :  current_commit_id,
                  'data_path' :  data_dir,
                  'tensorboard_folder_path' :  tensorboard_folder_path,
                  'keras_tuner_folder_path' :  keras_tuner_path,
                  'search_duration_seconds' : 0
                }

# #insert tuner search
insert_search(tuner_search, database_connection_details)

best_trials = tuner2.oracle.get_best_trials(num_models)
best_models = tuner2.get_best_models(num_models)

for model, trial in zip(best_models, best_trials):
    kt_trial_id = trial.trial_id

    #compute metrics
    hyperparams = trial.hyperparameters
    model_path = os.path.join(model_folder_path, kt_trial_id)
    model.save(model_path)

    val_results = model.evaluate(x = exp1_val_mris_preprocessed, y=exp1_val_label )
    val_loss, val_acc = val_results[0], val_results[1]
    train_results = model.evaluate(x = exp1_train_mris_preprocessed, y=exp1_train_label )
    train_loss, train_acc = train_results[0], train_results[1]

    #gathering into one dict

    trial = {
              'val_loss' : val_loss,
              'val_acc' : val_acc,
              'train_loss' : train_loss,
              'train_acc' : train_acc,
              'kt_trial_id'  : kt_trial_id,
              'search_id' : search_id, 
              'model_path': model_path,
              'learning_rate' : hyperparams.values['lr']
            }

    ##insert to DB##
    insert_trial(trial, database_connection_details)
    # Check to see if a trial_uid is generated

INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/ME Project/New idea/Results/Experiment 1/Normal loss/0c1420a1-af54-4101-9182-e741fe936b72/Models/0/assets
313/313 [==============================] - 33s 105ms/step - loss: 2.7389e-04 - accuracy: 1.0000 - categorical_crossentropy: 2.7389e-04
